## Downloading required dataset:

In [24]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip
!unzip collection5.zip -d Collection5
!rm -rf collection5.zip

--2024-07-08 11:41:30--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.04MB/s    in 0.9s    

2024-07-08 11:41:31 (2.04 MB/s) - ‘collection5.zip’ saved [1899530/1899530]

Archive:  collection5.zip
   creating: Collection5/Collection5/
  inflating: Collection5/Collection5/001.ann  
  inflating: Collection5/Collection5/001.txt  
  inflating: Collection5/Collection5/002.ann  
  inflating: Collection5/Collection5/002.txt  
  inflating: Collection5/Collection5/003.ann  
  inflating: Collection5/Collection5/003.txt  
  inflating: Collection5/Collection5/004.ann  
  inflating: Collection5/Collection5/004.txt  
  inflating: Collection5/Collection5/005.ann  
  infl

## Importing libraries

In [40]:
from corus import load_ne5
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    Doc
)
import pandas as pd
from sklearn.metrics import f1_score

## Initialize dataset + needded functions

In [41]:
dir = 'Collection5/Collection5'
records = load_ne5(dir)
segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
predicted_labels = []
true_labels = []

## Filing predicted + true labels

In [42]:
for j in range(1000):
  text = next(records)
  doc = Doc(text.text)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)

  for i in range(len(text.spans)):
      true_labels.append(text.spans[i].type)
      if i < len(doc.ner.spans):
          if (doc.spans[i].type == 'LOC'):
            predicted_labels.append('GEOPOLIT')
          else:
            predicted_labels.append(doc.spans[i].type)
      else:
          predicted_labels.append('wrong')
print("Processing completed")
print(len(predicted_labels))
print(len(true_labels))

Processing completed
26409
26409


## Check f1-score

In [43]:
f1_score = f1_score(true_labels, predicted_labels, average='weighted')
print(f"F1-Score: {f1_score:.4f}")

F1-Score: 0.5692
